**Import necessary libraries**

In [108]:
import pandas as pd
import numpy as np
import urllib.request
import sys
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

**Now we need to import a library to read tables from the website. This library is developed by Josua Schmid**

In [109]:
from html_table_parser import HTMLTableParser

**Now that we have imported the library, we can read the tables from the website of interest. In this notebook, we will work with neighbos list of Toronto**

In [110]:
target = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

**The following lines of code read the table from the wikipedia page**

In [111]:
req = urllib.request.Request(url=target)
f = urllib.request.urlopen(req)
xhtml = f.read().decode('utf-8')

In [112]:
p = HTMLTableParser()
p.feed(xhtml)

In [113]:
Table_list = p.tables

In [114]:
Table_data = pd.DataFrame.from_dict(Table_list[0])

In [115]:
Table_data.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


**As we can see from the dataframe above, the column headers are numbers and we need to change it to "Postcode", "Borough", and "Neighbourhood".**

In [116]:
new_header = Table_data.iloc[0] #grab the first row for the header
Table_data = Table_data[1:] #take the data less the header row
Table_data.columns = new_header #set the header row as the df header

In [117]:
Table_data.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


**Now we need to remove the rows that has not assigned Borough**

In [118]:
indexNames = Table_data[ Table_data['Borough'] == 'Not assigned' ].index
Table_data.drop(indexNames , inplace=True)
Table_data = Table_data.reset_index(drop=True)

In [119]:
Table_data.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [120]:
Table_data.shape

(210, 3)

In [121]:
Table_new = Table_data.groupby('Postcode').agg({'Borough':'first','Neighbourhood':','.join}).reset_index()

In [122]:
Table_new.head()
Postcode = Table_new['Postcode']

In [123]:
Table_new.shape

(103, 3)

In [124]:
Table_new.sort_values(by=['Postcode'])
Table_new.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Now we need to add the latitude and longitude of each region to the table. To do so, I read these infromation from the csv file provided. Then, I will join these two data frames**

In [125]:
geospatial = pd.read_csv('Geospatial_Coordinates.csv')
geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [126]:
Final_table = pd.merge(left=Table_new, right=geospatial, how='inner', left_on='Postcode', right_on='Postal Code')
Final_table.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [127]:
Final_table.drop(['Postal Code'], axis = 1)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [142]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Final_table['Latitude'], Final_table['Longitude'], Final_table['Borough'], Final_table['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
# set number of clusters
kclusters = 5

Final_table_clustering = Final_table.drop(['Postcode','Postal Code','Borough','Neighbourhood'], axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Final_table_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

In [130]:
Final_table.insert(0, 'Cluster Labels', kmeans.labels_)

In [134]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Final_table['Latitude'], Final_table['Longitude'], Final_table['Neighbourhood'], Final_table['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters